In [1]:
# Importation des modules
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import seaborn as sns

# Définir les options d'affichage pour afficher toutes les lignes sans troncature
pd.set_option("display.max_rows",None)
pd.set_option("display.max_colwidth",None)

# Modifier les options d'affichage pour limiter les décimales
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# BDD Title_basic

In [2]:
# Chargement de la Bdd et affichage 
title_basic = pd.read_csv("data/title.basics.tsv", sep="\t")
print(title_basic.info())
title_basic.head()

C:\Users\titgr\AppData\Local\Temp\ipykernel_22976\1597770290.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  title_basic = pd.read_csv("data/title.basics.tsv", sep="\t")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10858256 entries, 0 to 10858255
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 745.6+ MB
None


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,5,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


### NaN

In [3]:
# Transformer les valeurs \N en NaN
title_basic = title_basic.replace("\\N", np.nan)

##### NaN par colonne

In [4]:
# Initialisation des comptes de NaN
nan_counts_per_column = {}
total_nan_counts = 0

# Boucle sur chaque colonne et comptage des NaN
for column in title_basic.columns:
    nan_count = title_basic[column].isna().sum()
    nan_counts_per_column[column] = nan_count
    total_nan_counts += nan_count

# Affichage des résultats
print("Nombre de NaN par colonne:")
for column, count in nan_counts_per_column.items():
    print(f"Colonne {column}: {count} NaN")

print("\nNombre total de NaN dans tout le DataFrame:")
print(total_nan_counts)

Nombre de NaN par colonne:
Colonne tconst: 0 NaN
Colonne titleType: 0 NaN
Colonne primaryTitle: 19 NaN
Colonne originalTitle: 19 NaN
Colonne isAdult: 1 NaN
Colonne startYear: 1411924 NaN
Colonne endYear: 10732831 NaN
Colonne runtimeMinutes: 7495696 NaN
Colonne genres: 484258 NaN

Nombre total de NaN dans tout le DataFrame:
20124748


#### Création du dataframe titleBasic_clean

In [5]:
# Supprimer la colonne endYear, car \N 100.0 et pas très utile
titleBasic_clean = title_basic.drop(['endYear'], axis = 1)

In [6]:
titleBasic_clean.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,5,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,1,"Comedy,Short"


In [7]:
# Affichage des noms de l'ensemble des colonnes 
print(titleBasic_clean.columns)

Index(['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult',
       'startYear', 'runtimeMinutes', 'genres'],
      dtype='object')


##### Modification des noms des colonnes 

In [8]:
# Création du dictionnaire associant les anciens noms aux nouveaux noms de colonnes
dictionnaire_colonne_title_basic2 = {
    "tconst": "ID_title",
    "titleType": "title_type",
    "primaryTitle": "title",
    "originalTitle": "original_title",
    "isAdult": "isAdult",
    "startYear": "release_year",
    "runtimeMinutes": "runtime_minutes",
    "genres": "genres"
}

# On renomme les variables grâce à la méthode rename
titleBasic_clean = titleBasic_clean.rename(dictionnaire_colonne_title_basic2, axis = 1)

# Affichage info
titleBasic_clean.sample()

,ID_title,title_type,title,original_title,isAdult,release_year,runtime_minutes,genres
6167023,tt25396732,tvEpisode,Episode #1.87,Episode #1.87,0,2022,NaN,Drama


##### Conversion des colonnes en numériques

In [15]:
# Conversion de la colonne 'runtime_minutes' en entiers numérique, en gérant les valeurs non numériques (NaN)
def convert_to_int(val):
    try:
        return int(float(val))
    except (ValueError, TypeError):
        return np.nan

# Appliquer la fonction à la colonne 'release_year'
titleBasic_clean['runtime_minutes'] = titleBasic_clean['runtime_minutes'].apply(convert_to_int)

# Vérifiez le type de la colonne
titleBasic_clean['runtime_minutes'] = titleBasic_clean['runtime_minutes'].astype('Int64')

In [12]:
# Fonction pour convertir en entier, en gérant les valeurs non numériques
def convert_to_int(val):
    try:
        return int(float(val))
    except (ValueError, TypeError):
        return np.nan

# Appliquer la fonction à la colonne 'release_year'
titleBasic_clean['release_year'] = titleBasic_clean['release_year'].apply(convert_to_int)

# Vérifiez le type de la colonne
titleBasic_clean['release_year'] = titleBasic_clean['release_year'].astype('Int64')

In [13]:
# Obtenir la valeur minimale et maximale de la colonne 'release_Year'
min_year = titleBasic_clean['release_year'].min()
max_year = titleBasic_clean['release_year'].max()

# Afficher les résultats
print(f"Valeur minimale de 'release_year': {min_year}")
print(f"Valeur maximale de 'release_year': {max_year}")

Valeur minimale de 'release_year': 1874
Valeur maximale de 'release_year': 2031


##### Sélection à partir de la colonne isAdult

In [16]:
# Affichage des nombres d'occurences par modalités pour la colonne "isAdult"
titleBasic_clean['isAdult'].value_counts()

isAdult
0       10445336
1         346882
0          63463
1           2072
1980          63
1978          51
1985          46
1982          32
1972          29
2015          28
1974          25
1984          23
1983          22
2016          20
1981          18
2017          17
1973          15
1979          13
1987          13
2020           9
2018           9
1977           9
2013           8
2019           7
2014           6
2023           6
1966           5
1988           5
1986           4
1968           3
2022           2
1970           2
1971           2
1975           2
1967           1
1964           1
1969           1
2012           1
1976           1
2021           1
2011           1
2005           1
Name: count, dtype: int64

In [17]:
# Conserver les lignes où isAdult = 0
titleBasic_clean = titleBasic_clean[titleBasic_clean['isAdult'] == 0]

In [18]:
# Supprimer la colonne isAdult
titleBasic_clean = titleBasic_clean.drop(['isAdult'], axis = 1)

In [19]:
titleBasic_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10445336 entries, 0 to 10858255
Data columns (total 7 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   ID_title         object
 1   title_type       object
 2   title            object
 3   original_title   object
 4   release_year     Int64 
 5   runtime_minutes  Int64 
 6   genres           object
dtypes: Int64(2), object(5)
memory usage: 657.5+ MB


##### Sélection de la modalité "movie" dans la colonne title_type

In [20]:
# Sélection des occurences de la colonne title_type

    # Nombre d'occurence 
print(titleBasic_clean["title_type"].value_counts())

title_type
tvEpisode       8032943
short            993014
movie            671962
tvSeries         261470
video            187672
tvMovie          147418
tvMiniSeries      54371
tvSpecial         47622
videoGame         38561
tvShort           10302
tvPilot               1
Name: count, dtype: int64


In [21]:
# Selection des lignes == movie de la colonne title_type
titleBasic_clean = titleBasic_clean[titleBasic_clean["title_type"] == "movie"]

In [22]:
# Supprimer la colonne title_type
titleBasic_clean = titleBasic_clean.drop(columns= "title_type", axis=1)

##### Sélection année dans la colonne release Year

In [23]:
# Selection de la colonne release Year pour avoir film >= 1985 (Choix arbitraire, à justifier)
titleBasic_clean = titleBasic_clean[titleBasic_clean['release_year'] >= 1985]

In [25]:
print(titleBasic_clean.info())
titleBasic_clean.head(3)

<class 'pandas.core.frame.DataFrame'>
Index: 391474 entries, 11634 to 10858206
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   ID_title         391474 non-null  object
 1   title            391472 non-null  object
 2   original_title   391472 non-null  object
 3   release_year     391474 non-null  Int64 
 4   runtime_minutes  300252 non-null  Int64 
 5   genres           364005 non-null  object
dtypes: Int64(2), object(4)
memory usage: 21.7+ MB
None


,ID_title,title,original_title,release_year,runtime_minutes,genres
11634,tt0011801,Tötet nicht mehr,Tötet nicht mehr,2019,<NA>,"Action,Crime"
13079,tt0013274,Istoriya grazhdanskoy voyny,Istoriya grazhdanskoy voyny,2021,94,Documentary
15174,tt0015414,La tierra de los toros,La tierra de los toros,2000,60,NaN


## Gestion des Doublons

In [26]:
# Initialisation d'un dictionnaire pour stocker le nombre de doublons par colonne
duplicate_counts = {}

# Pour chaque colonne du DataFrame
for col in titleBasic_clean.columns:
    # Utilisation de duplicated() pour détecter les doublons
    num_duplicates = titleBasic_clean[col].duplicated(keep=False).sum() - titleBasic_clean[col].duplicated(keep='first').sum()
    duplicate_counts[col] = num_duplicates

# Affichage du nombre de doublons par colonne
for col, count in duplicate_counts.items():
    print(f"Colonne '{col}' a {count} doublons")

Colonne 'ID_title' a 0 doublons
Colonne 'title' a 20342 doublons
Colonne 'original_title' a 17875 doublons
Colonne 'release_year' a 44 doublons
Colonne 'runtime_minutes' a 359 doublons
Colonne 'genres' a 993 doublons


La colonne ID_title n'a pas de doublon. 

In [30]:
# Afficher les lignes qui sont des doublons dans la colonne 'original_title'
duplicates = titleBasic_clean[titleBasic_clean.duplicated('title')]

# Afficher les doublons pour la colonne 'original_title'
duplicates.head()

,ID_title,title,original_title,release_year,runtime_minutes,genres
87400,tt0089378,Making Contact,Joey,1985,98,"Action,Adventure,Drama"
88586,tt0090590,Adela,Adela,1987,92,Thriller
89290,tt0091312,Jwala,Jwala,1986,<NA>,"Action,Drama,Thriller"
89451,tt0091475,Mania: The Intruder,Mania,1986,87,"Drama,Horror,Mystery"
89571,tt0091598,Naam,Naam,1986,177,"Action,Crime,Drama"


In [31]:
# Compter les occurrences d'un titre dans la colonne 'original_title'
occurence_counts = titleBasic_clean['title'].value_counts().get('Adela', 0)
occurence_counts

4

In [33]:
df_Adela = titleBasic_clean[titleBasic_clean['title'] == 'Adela']
df_Adela

,ID_title,title,original_title,release_year,runtime_minutes,genres
86704,tt0088669,Adela,Adela,1985,80,Drama
88586,tt0090590,Adela,Adela,1987,92,Thriller
234692,tt0245066,Adela,Adela,2000,103,Thriller
2112644,tt1206475,Adela,Adela,2008,88,Drama


Les versions sorties après 1985 sont sans doute des versions remasterisées. On conserve uniquement la version original, donc la 1ère sortie.

In [34]:
# Supprimer les lignes en double dans la colonne 'title', en conservant explicitement la première occurrence
titleBasic_clean = titleBasic_clean.drop_duplicates(subset='title', keep='first')

In [35]:
df_Adela_clean = titleBasic_clean[titleBasic_clean['title'] == 'Adela']
df_Adela_clean

,ID_title,title,original_title,release_year,runtime_minutes,genres
86704,tt0088669,Adela,Adela,1985,80,Drama


In [36]:
# Compter les occurrences du mot 'Adela' dans la colonne 'original_title'
occurence_count_original_title = titleBasic_clean['original_title'].value_counts().get('Adela', 0)
occurence_count_original_title

1

In [37]:
# Initialisation d'un dictionnaire pour stocker le nombre de doublons par colonne
duplicate_counts = {}

# Pour chaque colonne du DataFrame
for col in titleBasic_clean.columns:
    # Utilisation de duplicated() pour détecter les doublons
    num_duplicates = titleBasic_clean[col].duplicated(keep=False).sum() - titleBasic_clean[col].duplicated(keep='first').sum()
    duplicate_counts[col] = num_duplicates

# Affichage du nombre de doublons par colonne
for col, count in duplicate_counts.items():
    print(f"Colonne '{col}' a {count} doublons")

Colonne 'ID_title' a 0 doublons
Colonne 'title' a 0 doublons
Colonne 'original_title' a 1302 doublons
Colonne 'release_year' a 44 doublons
Colonne 'runtime_minutes' a 351 doublons
Colonne 'genres' a 970 doublons


In [38]:
# Afficher les lignes qui sont des doublons dans la colonne 'original_title'
duplicates = titleBasic_clean[titleBasic_clean.duplicated('original_title')]

# Afficher les doublons pour la colonne 'original_title'
duplicates.head()

,ID_title,title,original_title,release_year,runtime_minutes,genres
87400,tt0089378,Making Contact,Joey,1985,98,"Action,Adventure,Drama"
89451,tt0091475,Mania: The Intruder,Mania,1986,87,"Drama,Horror,Mystery"
95239,tt0097414,Money,Geld,1989,98,Comedy
96096,tt0098284,Four Loves,Si qian jin,1989,97,"Comedy,Drama,Romance"
96112,tt0098300,Shag,Shag,1988,98,"Comedy,Drama,Romance"


In [39]:
# Compter les occurrences d'un titre dans la colonne 'original_title'
occurence_counts = titleBasic_clean['original_title'].value_counts().get('Joey', 0)
occurence_counts

2

In [40]:
df_Joey= titleBasic_clean[titleBasic_clean['original_title'] == 'Joey']
df_Joey

,ID_title,title,original_title,release_year,runtime_minutes,genres
87399,tt0089377,Joey,Joey,1986,97,Drama
87400,tt0089378,Making Contact,Joey,1985,98,"Action,Adventure,Drama"


In [41]:
df_Siqianjin = titleBasic_clean[titleBasic_clean['original_title'] == 'Si qian jin']
df_Siqianjin

,ID_title,title,original_title,release_year,runtime_minutes,genres
93971,tt0096105,Si qian jin,Si qian jin,1988,<NA>,NaN
96096,tt0098284,Four Loves,Si qian jin,1989,97,"Comedy,Drama,Romance"


In [42]:
# Supprimer les lignes en double dans la colonne 'original_title', en conservant l'occurence qui n'a pas de NaN dans la ligne

# Classer de manière à ce que les lignes sans NaN viennent en premier
titleBasic_clean['na_count'] = titleBasic_clean.isna().sum(axis=1)
titleBasic_clean = titleBasic_clean.sort_values(by='na_count')

# Supprimer les lignes en double en gardant la première occurrence (lignes sans NaN seront prioritaires)
titleBasic_clean = titleBasic_clean.drop_duplicates(subset='original_title')

# Retirer la colonne temporaire utilisée pour le classement
titleBasic_clean = titleBasic_clean.drop(columns=['na_count'])

titleBasic_clean.info()

C:\Users\titgr\AppData\Local\Temp\ipykernel_22976\3431865491.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  titleBasic_clean['na_count'] = titleBasic_clean.isna().sum(axis=1)


<class 'pandas.core.frame.DataFrame'>
Index: 348394 entries, 4881963 to 348953
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   ID_title         348394 non-null  object
 1   title            348393 non-null  object
 2   original_title   348393 non-null  object
 3   release_year     348394 non-null  Int64 
 4   runtime_minutes  266753 non-null  Int64 
 5   genres           323317 non-null  object
dtypes: Int64(2), object(4)
memory usage: 19.3+ MB


In [43]:
df_Siqianjin = titleBasic_clean[titleBasic_clean['original_title'] == 'Si qian jin']
df_Siqianjin

,ID_title,title,original_title,release_year,runtime_minutes,genres
96096,tt0098284,Four Loves,Si qian jin,1989,97,"Comedy,Drama,Romance"


#### Gestion des valeurs manquantes

In [45]:
# Calcul du nombre total de lignes contenant au moins une valeur manquante
total_missing_rows_sum = titleBasic_clean.isna().any(axis=1).sum()

# Calcul du nombre total de colonnes contenant au moins une valeur manquante
total_missing_columns_sum = titleBasic_clean.isna().any(axis=0).sum()

In [46]:
# Afficher le nombre total de lignes avec des valeurs manquantes
print(f"Nombre total de lignes contenant des valeurs manquantes : {total_missing_rows_sum}")

# Afficher le nombre total de colonnes avec des valeurs manquantes
print(f"Nombre total de colonnes contenant des valeurs manquantes : {total_missing_columns_sum}\n")

Nombre total de lignes contenant des valeurs manquantes : 92520
Nombre total de colonnes contenant des valeurs manquantes : 4



In [49]:
# Identifier les colonnes avec des valeurs manquantes
columns_with_missing_values = titleBasic_clean.columns[titleBasic_clean.isna().any()].tolist()

# Afficher les colonnes avec des valeurs manquantes
print("Colonnes avec des valeurs manquantes:", columns_with_missing_values)

Colonnes avec des valeurs manquantes: ['title', 'original_title', 'runtime_minutes', 'genres']


On ne retire pas les lignes avec NA issue de ces colonnes car on va remplacer les valeurs manquantes avec le DF TMDB, pour 
minimiser la perte en données.

In [48]:
# # Détecter et afficher les lignes contenant des valeurs manquantes
# rows_with_missing_values = titleBasic_clean[titleBasic_clean.isna().any(axis=0)]

# # Si vous voulez afficher ces lignes
# #print(rows_with_missing_values)

In [50]:
titleBasic_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 348394 entries, 4881963 to 348953
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   ID_title         348394 non-null  object
 1   title            348393 non-null  object
 2   original_title   348393 non-null  object
 3   release_year     348394 non-null  Int64 
 4   runtime_minutes  266753 non-null  Int64 
 5   genres           323317 non-null  object
dtypes: Int64(2), object(4)
memory usage: 19.3+ MB


In [51]:
# 1-Sauvegarder en format CSV
titleBasic_clean.to_csv('titleBasic_clean.csv', index=False)